## Converting data to documents
This notebook is for using the data we obtained in the dataframe and representing it in a langchain Document object. This is a data structure very applicable to Data Science and NLP tasks, since it allows us to separate the actual text content we want to use for our embeddings from the metadata tags, such as source and url. 

### Part 1: Importing essential packages

In [2]:
import json
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
import sys
import openai
from openai import OpenAI
from langchain.chat_models import ChatOpenAI

### Part 1 - Loading the df from memory

Since we already extracted the data in the scraping notebook, we can just load it from memory.

In [3]:
df = pd.read_json('data.json', orient='records', lines=True)

### Part 2 - The langchain Document

Now, the idea behind this data structure is that it is very suitable for NLP tasks. It has one attribute, page_content, that is the plain text we intend to embed and store in a vectorized form. Additionally, it has a metadata field which is a customizable dictionary where we can store whatever we want. This content is not touched by the embedding model, so we can use this downstream for filtering etc.

This function simply loops through the dataframe and builds a Document instance. This is where you would typically introduce as much metadata as you can find, since it is cheap in terms of storage and gives you much more dynamic possibilities in future alterations.

In [4]:

def df_to_langchain_documents(df):
    """
    Convert a pandas DataFrame into a list of LangChain Document objects.

    Parameters:
    df (pd.DataFrame): The DataFrame to convert.

    Returns:
    list: A list of LangChain Document objects.
    """
    documents = []
    for _, row in df.iterrows():
        doc = Document(
            page_content=row['text'],
            metadata={
                'key': row['key'],
                'url': row['url'],
                'category': row['category']
            }
        )
        documents.append(doc)
    return documents

In [5]:
documents = df_to_langchain_documents(df)

In [7]:
print(documents[0])

page_content='Nyfiken På We Know IT? | Om Oss Våra kunder Våra tjänster Konsultuthyrning Webbutveckling Apputveckling UX/UI - Design Digital marknadsföring Hosting & förvaltning Om oss Karriär Kontakta oss Våra kunder Våra tjänster Konsultuthyrning Webbutveckling Apputveckling UX/UI - Design Digital Marknadsföring Hosting & Förvaltning Om oss Karriar Kontakta oss Varför We Know IT? Vi är IT-konsultbolaget som satsar på studerande och nyexade talanger med höga ambitioner och drivkrafter.\xa0Experter på utveckling, design och digital strategi, We Know IT helt enkelt. \u200bMål, vision & sådant gött Vårt mål är att vara det självklara valet för studenter att starta sin karriär på, och det självklara valet för företag som behöver motiverad och innovativ kompetens. Det är inte alla som får chansen att jobba med morgondagens skarpaste konsulter. Vi får det varje dag, och du som kund eller samarbetspartner får stora möjligheter genom att välja oss. Vi vill ha kul på vägen och leverera över fö

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
chunked_docs = text_splitter.split_documents(documents)

In [8]:
current_key = None
counter = 0

for chunk in chunked_docs:
    key = chunk.metadata.get("key")
    if key != current_key:
        current_key = key
        counter = 0
    chunk.metadata["chunk_id"] = f"{key}_{counter}"
    counter += 1


### Part 3 - The embedding functions
These are supporting functions, based on the already existing functions and methods on HuggingFace. It can be nice to write these manually in order to have more control of what we do with the text chunks. 

In [3]:
from typing import Optional


def doc_embedding(embedding_model: str, 
                  model_kwargs: dict={'device':'cpu'}, 
                  encode_kwargs: dict={'normalize_embeddings':True},
                  cache_folder: Optional[str]=None,
                  multi_process: bool=False
                  ) -> HuggingFaceEmbeddings:
    embedder = HuggingFaceEmbeddings(
        model_name = embedding_model,
        model_kwargs = model_kwargs,
        encode_kwargs = encode_kwargs,
        cache_folder = cache_folder,
        multi_process = multi_process
    )
    return embedder

def get_API_embedding(text, model):
    embedder = doc_embedding(model)
    embedding = embedder.embed_query(text)
    return embedding

### Part 4 - Create the vector databases

In order to do this, we need to download an embedding model. In this example, we download the model locally, since it is rather small. The task is then to use our created langchain Documents and the embedding models to locally persist a directory, which is the Chroma collection and the Chroma DB we will then use for context retrieval. 

In [4]:
#model = "intfloat/multilingual-e5-large-instruct"
model = "mixedbread-ai/mxbai-embed-large-v1"
embedding_model = doc_embedding(model)

persist_directory = "e5_ml_db"

# vectordb = Chroma.from_documents(documents=chunked_docs, 
#                                  embedding=embedding_model, 
#                                  persist_directory=persist_directory)

/Users/axelandersson/Desktop/github/wkit-chatbot/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/axelandersson/Desktop/github/wkit-chatbot/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
vectordb = Chroma(embedding_function=embedding_model, persist_directory=persist_directory)

In [6]:
query = "Webbapplikationer som byggts genom åren"
context = vectordb.similarity_search_with_relevance_scores(query, 5)

In [7]:
load_dotenv()
# Retrieve the value of the environment variable
openai_key = os.environ.get("OPENAI_KEY")

In [8]:
chat_model = ChatOpenAI(
    openai_api_key=os.environ.get("OPENAI_KEY"),
    model='gpt-3.5-turbo-1106',
    temperature=0.1
)

/Users/axelandersson/Desktop/github/wkit-chatbot/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
from langchain.schema import (SystemMessage, HumanMessage, AIMessage)

system_prompt = """Du är en hjälpsam AI assistent, specialiserad på att svara på frågor om ett IT-konsultbolag som
                heter We Know IT. Du kommer att få frågor samt utvald information, vilken du kan använda
                för att svara på frågan. Svara på svenska."""

def get_prompt(query: str, vectordb):
    # Retrieve 10 chunks with relevance scores
    context_results = vectordb.similarity_search_with_relevance_scores(query, 10)
    
    # Extract the page_content from each context document
    context = "\n".join([doc.page_content for doc, score in context_results])
    
    # Construct the final prompt
    user_prompt = f"""Svara på följande fråga: {query}.

Du kan använda följande information för att generera ditt svar:
{context}"""

    return user_prompt


In [13]:
test = get_prompt("Hur bygger We Know It sina webbapplikationer?", vectordb)


In [15]:
query = "Hur bygger We Know It sina webbapplikationer?"
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=get_prompt(query, vectordb))
]
response = chat_model.invoke(messages)
print(response.content)

We Know IT bygger sina webbapplikationer genom en effektiv utvecklingsfas där de använder sitt unika ramverk för att säkerställa högsta kvalitet och kontinuerlig rapportering. De lägger stor vikt vid att SEO-anpassa allt innehåll i termer av sökordsdensitet, ordmängd och kvalitet. När webbutvecklingen är färdig och det nya innehållet har integrerats, genomförs kvalitetssäkring och lansering av webbplatsen. We Know IT fokuserar också på konverteringsoptimering för att förbättra användarupplevelsen och öka konverteringar på webbplatsen.
